## [한국수자원공사] 가뭄 수위관측 openAPI

##### 목적 : 수위 측정 관측소별 3개년 데이터 수집

In [1]:
### 사용 라이브러리 선언
import pandas as pd
import requests
import datetime as dt

from bs4 import BeautifulSoup
from urllib.parse import urlencode, quote_plus, unquote

### 1. XML API 요청을 위한 URL 생성 (openAPI 참조문서의 요청명세를 확인하여 필수 queryString 구성 및 인코딩)
## 가뭄 수위관측 openAPI 조회를 위한 queryString 필수 값으로 관측 시설물 코드 리스트 생성
facilityCode = pd.read_csv("../dataset/facilityCode_1003_ms949.csv")
facilityCodeList = facilityCode.cd.tolist()
facilityNameList = facilityCode.cdnm.tolist()

baseUrl = "http://apis.data.go.kr/B500001/drghtWlobsOper2/operInfoList2"
currentYear = dt.datetime.now().year
yearList = [str(currentYear-3), str(currentYear-2), str(currentYear-1)] # 이전 3개년 검색에 필요한 데이터
serviceKey = "2AlYmSdbDUuKGC0DivNISdLDUqJewAoneYBBTMwCjqcQz8cIiQXSp0je68IfpEVPg6+J2f8OeNy3lak986T1rQ=="
numOfRows = "9999"
pageNo = "1"
for i in range (0, len(facilityCodeList)): # 각각의 관측소를 기준으로 데이터프레임을 생성하고자 함
    wlobsCd = facilityCodeList[i]
    region = facilityNameList[i] # 데이터프레임 csv저장시 사용할 데이터
    for j in range(0, len(yearList)):
        year = yearList[j]
        queryParam2 = '?' + urlencode(
            {
                quote_plus('serviceKey') : serviceKey, 
                quote_plus('numOfRows') : numOfRows, # 한 페이지에 노출되는 결과 값 (관측소에서 일일 데이터 측정, 따라서 연간 최대 365개)
                quote_plus('pageNo') : pageNo, 
                quote_plus('wlobsCd') : wlobsCd, # 관측소코드 값
                quote_plus('stDt') : year + "0101", 
                quote_plus('edDt') : year + "1231"
            }
        )
        obsTargetUrl = baseUrl + queryParam2 

        
        ### 2. XML API 요청 및 응답받기(BeautifulSoup으로 편집)
        resp = requests.get(obsTargetUrl)
        resp.encoding = "utf-8"
        xml = resp.text
        bsXml = BeautifulSoup(xml, "lxml-xml")
        
        ### 3. DataFrame으로 만들 필요한 데이터 수집
        ## 3-1.
        rows = bsXml.findAll(name = "item")

        ## 사용할 데이터 선언
        rowList = [] # 전체 행을 담을 리스트(DataFrame 생성시 사용)
        columnDic = {} # 각 행의 컬럼명과 컬럼값을 한쌍으로 담을 딕셔너리

        ### 3-2. 행별 컬럼값을 추출(딕셔너리를 사용하여 컬럼명에 컬럼값을 매칭하여 컬럼값이 밀리는 현상 방지)
        for k in range (0, len(rows)):

            columns = rows[k].findAll()

            for l in range (0, len(columns)):

                columnName = columns[l].name

                columnValue = columns[l].text

                columnDic[columnName] = columnValue

            rowList.append(columnDic)
            columnDic =  {}
            
        ### 4. DataFrame 생성 및 csv저장
        waterLevelInfoDf = pd.DataFrame(rowList)
        waterLevelInfoDf.to_csv(("../dataset/waterLevel/waterLevelInfoDf_{}_{}_{}.csv".format(wlobsCd, region, year)), index = False, encoding = "ms949")